In [164]:
import praw
import os
import configparser
import webbrowser
import random
import json
import pandas as pd
import numpy as np
import collections
import string
from nltk.corpus import stopwords
from nltk.stem import *
os.chdir("C:/Users/macle/Desktop/UPC Masters/Semester 2/CI/Final Project")

In [128]:

config = configparser.ConfigParser()
config.read('secrets.ini')

reddit_client_id = config.get('reddit', 'client_id')
reddit_api_key = config.get('reddit', 'api_key')
r.set_oauth_app_info(client_id=reddit_client_id,
                      client_secret=reddit_api_key,
                      redirect_uri='http://cole-maclean.github.io/MAI-CI/')

In [129]:
url = r.get_authorize_url('uniqueKey', 'identity', True)
webbrowser.open(url)

True

In [130]:
access_information = r.get_access_information('B3w4W9k5xFbJpaLv8Xpuf-iQj34')
r.set_access_credentials(**access_information)

In [131]:
authenticated_user = r.get_me()
print(authenticated_user.name, authenticated_user.link_karma)

upcmaici 1


In [142]:
with open('reddit_data.json','r') as data_file:    
    reddit_data = json.load(data_file)
with open('scrapped_users.json','r') as data_file:    
    scrapped_users = json.load(data_file)
for i in range(10):
    print ("scrapping " + str(i) + "th subreddit")
    rand_submission = r.get_random_submission()
    print ("scrapping users from subreddit r/" + rand_submission.subreddit.display_name)
    if len(rand_submission.comments) >=3:
        rnd_comments = random.sample(rand_submission.comments,min(10,len(rand_submission.comments)))
        for comment in rnd_comments:
            if isinstance(comment, praw.objects.Comment):
                user = comment.author
                if user:
                    if user.name in scrapped_users:
                        print ('user ' + user.name + 'already scraped')
                    else:
                        scrapped_users.append(user.name)
                        for user_comment in user.get_comments(limit=None,_use_oauth=False):
                            body = user_comment.body.split()
                            if len(body) >= 10:
                                rand_words = random.sample(body,10)
                            else:
                                rand_words = body
                            reddit_data.append([user_comment.subreddit.display_name,
                                                                              user_comment.created_utc,rand_words])
                    
with open('reddit_data.json','w') as data_file:
    json.dump(reddit_data, data_file)
with open('scrapped_users.json','w') as data_file:
    json.dump(scrapped_users, data_file)

scrapping 0th subreddit
scrapping users from subreddit r/news
scrapping 1th subreddit
scrapping users from subreddit r/videos
scrapping 2th subreddit
scrapping users from subreddit r/grimfandango
scrapping 3th subreddit
scrapping users from subreddit r/streetwear
scrapping 4th subreddit
scrapping users from subreddit r/Foodforthought
scrapping 5th subreddit
scrapping users from subreddit r/hearthstone
scrapping 6th subreddit
scrapping users from subreddit r/aussievapers
scrapping 7th subreddit
scrapping users from subreddit r/technology
scrapping 8th subreddit
scrapping users from subreddit r/dirtykikpals
scrapping 9th subreddit
scrapping users from subreddit r/futurama


In [147]:
df = pd.DataFrame(reddit_data,columns=['subreddit','utc_stamp','rnd_words'])
df.head()

,subreddit,utc_stamp,rnd_words
0,breathinginformation,1479531643,"[beat, Griffon, the, ball, when, in, Peggle, d..."
1,baseball,1479531569,"[Hopefully, both, deals.]"
2,baseball,1479530961,"[Rodon, and, Gonzalez, 1-2., Shields, 3.]"
3,baseball,1479530947,"[I, count, Rodon, and, Miguel, Gonzalez, befor..."
4,breathinginformation,1479527248,"[Ryan, dropped, old, in, school., knife, Wow, ..."


In [152]:
f = {'subreddit':['count'], 'utc_stamp':[np.min,np.max],'rnd_words':['count']}
grouped = df.groupby('subreddit').agg(f)

In [ ]:
list_of_words = []
translate_table = dict((ord(char), None) for char in string.punctuation)
stemmer = PorterStemmer()
for user_comment in reddit_data:
    for word in user_comment[2]:
        if len(word) < 45:
            clean_word = word.translate(translate_table)
            wrd_stm = stemmer.stem(clean_word)
            list_of_words.append(wrd_stm)
word_counts = collections.Counter(list_of_words)

In [135]:
len(reddit_data.keys())

741